In [26]:
import json
from os.path import join
import pandas as pd
from pygments import highlight
from pygments.formatters import Terminal256Formatter
from pygments.lexers import PythonLexer
from pygments.styles import get_style_by_name
import pickle
from openai import OpenAI
import numpy as np



# def read_dir_data(base_path):
#     data = []
#     for dir_name in os.listdir(base_path):
#         dir_path = os.path.join(base_path, dir_name)
#         if os.path.isdir(dir_path):
#             record = {"directory": dir_name}
#             with open(os.path.join(dir_path, "metadata.json"), "r") as file:
#                 metadata = json.load(file)
#             record.update(metadata)
#             with open(os.path.join(dir_path, "question.txt"), "r") as file:
#                 record["question"] = file.read().strip()
#             solutions_path = os.path.join(dir_path, "solutions.json")
#             if os.path.isfile(solutions_path):
#                 with open(solutions_path, "r") as file:
#                     record["solutions"] = json.load(file)
#             data.append(record)
#     df = pd.DataFrame(data)
#     return df


# base_path = "train"
# df = read_dir_data(base_path)


with open(join('data_artifacts', 'train.pkl'), 'rb') as f:
    df = pickle.load(f)

In [ ]:
'''
# save the df as a pickle file
df.to_pickle(join("data_artifacts", "train.pkl"))

# tell me size in MB
print("Size of the dataframe in MB:", df.memory_usage(deep=True).sum() / 1024 ** 2)
'''

In [29]:
import asyncio

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.runnables.base import RunnableBinding
from langchain_openai import ChatOpenAI


def get_llm(model: str):
    if model == "cleverchain-gpt-4o":
        openai_gpt4_args = {
            "seed": 123,
            # "function_call": None,
            # "response_format": {"type": "json_object"},
        }
        return ChatOpenAI(
            model="gpt-4o-2024-05-13",
            temperature=0.5,
            max_tokens=2000,
            model_kwargs=openai_gpt4_args,
            openai_api_key='sk-proj-Zwf0tSiDacdeiQRTMLVHT3BlbkFJMbreGiCSIpGX1rwYEtHD'
        )




# Asynchronous function to process each problem description
async def process_description(question, solution):
    sys_msg = """
    Given a competitive programming problem description and its expert solution, simplify the description to focus strictly on computational and data structure terms. Remove any thematic or domain-specific language, abstracting the problem to its algorithmic essentials.

    Task:
    - Parse the input problem description to identify core computational processes and data handling.
    - Remove all thematic or narrative elements, replacing them with general computational terms that capture the essence of the problem without domain-specific context.
    - Output a streamlined version that uses only terms relevant to computer science, data structures, and algorithms, ensuring it is suitable for database indexing and algorithm lookup.


    Example:
    Input:
    Problem Description: 'We have a sandglass consisting of two bulbs, bulb A and bulb B. These bulbs contain some amount of sand. When we put the sandglass, either bulb A or B lies on top of the other and becomes the upper bulb. The other bulb becomes the lower bulb. The sand drops from the upper bulb to the lower bulb at a rate of 1 gram per second. When the upper bulb no longer contains any sand, nothing happens. Initially at time 0, bulb A is the upper bulb and contains a grams of sand; bulb B contains X-a grams of sand (for a total of X grams). We will turn over the sandglass at time r_1, r_2,..,r_K. Assume that this is an instantaneous action and takes no time. Here, time t refers to the time t seconds after time 0. You are given Q queries. Each query is in the form of (t_i,a_i). For each query, assume that a=a_i and find the amount of sand that would be contained in bulb A at time t_i.'

    Solution: Simplified version focusing on the essential algorithmic steps.

    Output:
    ```
    'Given two variables, A and B, holding a total of X units. Initially, A has 'a' units and B has 'X-a' units. Units transfer from the active variable to the passive variable at a rate of 1 unit per second. Switches between which variable is active occur at times r_1, r_2, ..., r_K. Answer Q queries, each specifying a time t_i and an initial amount 'a_i' for A. Determine the number of units in A at each specified time t_i.

    key DSA terms: "BFS, state transition......"
    ```

    This task will help in creating clear and direct problem statements for use in vector databases and algorithm lookup systems.

    Do not include input and output formats in the simplified version. Only focus on the core computational and data handling aspects of the problem. Ensure that the simplified version is concise and free of thematic or narrative elements. The output should be suitable for indexing in a vector database and RAG lookup system.

    Once again, it is very very important that you exclude jargon. Topical terms, proper nouns, hypotehtical names, prooblem lore, cannon, and other weird noise should always be removed. The goal is to make the problem statement as generic as possible, so that it can be used in a variety of contexts. This is a very important task, so please take your time and do it well.
    """
    system_message = SystemMessage(content=sys_msg)

    task = f"""
    ---

    This is the first competition problem, TASK 1:
    ```
    {question}
    ```

    This is the provided expert solution:
    ```
    {solution}
    ```
    """
    initial_human_message = HumanMessage(content=task)
    messages = [system_message, initial_human_message]
    return await LLM.ainvoke(messages)

LLM = get_llm("cleverchain-gpt-4o")

demo_testing = df[df["difficulty"] == "competition"]
questions = demo_testing["question"]
solutions = [sol[0] for sol in demo_testing["solutions"]]


In [30]:
tasks = []
for q, s in zip(questions, solutions):
    q = q.strip()    
    '''
    print("-*" * 40)
    print("-*" * 40)
    print("-*" * 40)
    print(q)
    print("-*" * 40)
    print(highlight(s, PythonLexer(), Terminal256Formatter(style='monokai')))
    print("-*" * 40)
    print("-*" * 40)
    print("-*" * 40)
    '''
    tasks.append(process_description(q, s))

In [31]:
results = await asyncio.gather(*tasks)

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-qd2wW1ZjR02UllQFwSfnne98 on tokens per min (TPM): Limit 30000, Used 28517, Requested 3456. Please try again in 3.946s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [24]:

simplified_descriptions = []
for res in results:
    simplified_descriptions.append(res.content)

dict = {'Problem Description': simplified_descriptions, 
        'Solution': solutions}

simp_df = pd.DataFrame(dict)

simp_df.to_pickle(join('data_artifacts', 'simplified_df.pkl'))

In [28]:
def generate_embedding(client, text):
    """
    Generate an embedding for the given text using the specified model from OpenAI.

    Parameters:
    client (OpenAI): The OpenAI client used to generate embeddings.
    text (str): The text to generate an embedding for.

    Returns:
    list: The embedding vector for the given text.
    """
    response = client.embeddings.create(
        input=text,
        model='text-embedding-3-small'
    )
    return response.data[0].embedding

open_ai_client = OpenAI(api_key ='sk-proj-Zwf0tSiDacdeiQRTMLVHT3BlbkFJMbreGiCSIpGX1rwYEtHD')

print(simp_df.columns)
embeddings = np.zeros((simp_df.shape[0], 1536))

for index, row in simp_df.iterrows():
    embeddings[index] = generate_embedding(open_ai_client, row['Problem Description'])

np.save(join('data_artifacts', 'problem_embeddings.npy'), embeddings)

Index(['Problem Description', 'Solution'], dtype='object')
